v1 à V2 -> on va deja voir si on prend un voxel dans la région 1 comme seed, ca marche sur les voxels de la region 1<br>
v2 à V3 -> on va  voir si on prend le signal moyen dans la région 1 comme seed, si ca marche sur les voxels de la region 1 <br>
v3 à v4 ->  on va  voir si on prend le signal moyen dans la région 1 comme seed, si ca marche sur les voxels de tout le cerveau <br>
v4 à v5 -> on dev la fct propre à intéger à aicha qui ne prend que 3 param= residus, moyenne des résidus sur une region, brain mask c1c2 normalizés

In [1]:
-

In [2]:
computeCorrelations('/scratch/user/hirsch/datadir4/data_results_py/functionnal/arima/arimaResidus/_report/arima_residu.nii.gz',
                    '/scratch/user/hirsch/datadir4/data_results_py/functionnal/regMeants/arima_residu_ts.txt',
                    '/scratch/user/hirsch/datadir4/data_results_py/structural/normalized_c1c2_file/wc1t0009_t1_s03_maths_thresh.nii')

(193436, 3)
580308


## fin fonction et test

In [6]:
# first we get the region 1 image we will need it later for calculation in the same region but not in the general case 
# where we compute gm & wm
regSeedFile = '/homes_unix/hirsch/_ipnotebooks/region_image.nii'
reg_data=nib.load(regSeedFile) 

In [7]:
# then we get the seed mean signal
seedFile = '/scratch/user/hirsch/datadir4/data_results_py/functionnal/regMeants/arima_residu_ts.txt'
seed_ts_array = np.loadtxt(seedFile)

In [8]:
# from an other hand we get the residus 4D matrix
fmri_file = '/scratch/user/hirsch/datadir4/data_results_py/functionnal/arima/arimaResidus/_report/arima_residu.nii.gz'
fmri_data=nib.load(fmri_file) 
fmri_array=np.asarray(fmri_data.dataobj)

In [10]:
# we get the coordinnates of voxels in all gm normalized todo get the wm also
regTargetFile = '/scratch/user/hirsch/datadir4/data_results_py/structural/normalized_files/wc1t0009_t1_s03.nii'
reg_data=nib.load(regTargetFile) 
regarray=np.asarray(reg_data.dataobj)
# transpose(nonzero(a))
reg_coords = np.transpose(np.nonzero(regarray))
volume_shape = reg_coords.shape
print volume_shape
coords = list(np.ndindex(volume_shape))
print len(coords)

(197787, 3)
593361


In [11]:
# the we iterate the correlation calculation on all voxels of region 1

# np.full(shape, fill_value, dtype=None, order='C')
# the correlation matrix is initialized with all values to 0 
corr_matrix = np.full(reg_data.shape, 0, dtype=float)

for i in range(reg_coords.shape[0]):
    target_array = fmri_array[reg_coords[i, 0], reg_coords[i, 1],reg_coords[i,2], :]
    #print target_array
    non_zero_nb = np.count_nonzero(target_array)
    if non_zero_nb:
        try:
            p = pearsonr(seed_ts_array,target_array) 
            if p[0] > 0.5:
                print p[0]
            corr_matrix[reg_coords[i, 0], reg_coords[i, 1],reg_coords[i,2]] = p[0] 
        except:
            print "exception"
    
# save matrix in a file
# create the resulting image
corr_image = nib.Nifti1Image(corr_matrix,affine=reg_data.affine, header=reg_data.header)
# save the correlation array
out_file = os.getcwd() + '/' + 'corr_reg1v4.nii'
nib.save(corr_image, out_file)
